In [1]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import getpass
import pandas as pd

In [3]:
client_id=str(getpass.getpass('enter your client id?')) #ee2037b6235f4d19a75129a108d45a28
client_secret=str(getpass.getpass('enter your clientsecret?')) #92bfc64c53914d54a30f44553174fe89

enter your client id?········
enter your clientsecret?········


In [4]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret))

# 1. Get a list of thousands songs with audio features

## 1.1 Define functions to extract audio features from one playlist (<100 tracks)

In [ ]:
def audiofeat_from_playlist(playlist_id):
    playlist = sp.playlist_tracks(playlist_id) #define playlist
    playlist_uris=[] # create empty list of uris 
    for item in playlist['items']: #append list with uri of each song in the playlist
        playlist_uris.append(item['track']['uri'])
    audio_features = sp.audio_features(tracks=playlist_uris) #extract audio features 
    return (audio_features)

# 1.2 Looping the function over a series of playlists

In [ ]:
playlists = ['37i9dQZF1DZ06evO39YXQI', '37i9dQZF1DX5EkyRFIV92g', '37i9dQZF1DX8ucPPHyzDUE', '37i9dQZF1DX55yuR78Invt',
           '37i9dQZF1DWWxPM4nWdhyI', '37i9dQZF1DZ06evO161ZV6', '37i9dQZF1DZ06evO2zJOi2', '37i9dQZF1DZ06evO2ZTWSp',
           '37i9dQZF1DZ06evO2csqsN', '37i9dQZF1DZ06evO2FvyO4', '37i9dQZF1DX2apWzyECwyZ', '37i9dQZF1DX9tzt7g58Xlh',
           '37i9dQZF1DZ06evO1q1llu', '37i9dQZF1DXc2aPBXGmXrt', '37i9dQZF1DZ06evO30JqGN', '37i9dQZF1DZ06evO3t7Mpz',
           '37i9dQZF1DX1whYo9Mg2Mt', '37i9dQZF1DZ06evO0Kc0i8', '37i9dQZF1DZ06evO1aBeik', '37i9dQZF1DZ06evO2oTcyc',
           '37i9dQZF1DZ06evO4kEIFJ', '37i9dQZF1DX7JVsTTEkyUT', '37i9dQZF1E4k8iFIrwwqkW', '37i9dQZF1E4BoMLQnbv68l',
           '37i9dQZF1E4BHGOS34BXBo', '37i9dQZF1E4mHJ8YzKWXiJ', '37i9dQZF1E4CWDuYF7GiUo']

In [ ]:
af_df = pd.DataFrame(audiofeat_from_playlist(playlists[0]))

In [ ]:
af_df

In [ ]:
for item in playlists[1:]:
    audio_features_playlist = audiofeat_from_playlist(item) 
    af_df_temp = pd.DataFrame(audio_features_playlist)
    af_df = pd.concat([af_df, af_df_temp])

In [ ]:
af_df.info()

# 1.3 improving the function audiofeat_from_playlist for playlist longer than 100 songs

# long playlist = '37i9dQZF1DWXRqgorJj26U' 

def get_playlist_tracks_long(playlist_id): #get tracks for playlists > 100
    playlist=sp.playlist_tracks(playlist_id)
    tracks=playlist['items']
    while playlist['next']:
        playlist=sp.next(playlist)
        tracks.extend(playlist['items'])
    return tracks

tracks = get_playlist_tracks_long('37i9dQZF1DWXRqgorJj26U')

tracks

audio_features_tot = []
playlist_uris_chunked = []
playlist_uris=[]

def audiofeat_from_playlistLONG(playlist_id):
    tracks = get_playlist_tracks_long(playlist_id)
    
    audio_features_tot = []
    playlist_uris_chunked = [] 
    playlist_uris=[]
    
    for item in tracks: #append list with uri of each song in the playlist
        playlist_uris.append(item['track']['uri'])
    
    
    chunk_size = 100
    for i in range(0, len(playlist_uris), chunk_size):
        playlist_uris_chunked.append(playlist_uris[i:i+chunk_size])
    
    for uri in playlist_uris_chunked:
        audio_features_temp = sp.audio_features(tracks=uri)
        audio_features_tot.append(audio_features_temp) #extract audio features
    
     
    af_df_1 = pd.DataFrame(audio_features_tot[0])
    
    for item in audio_features_tot[1:]:
        af_df_temp_1 = pd.DataFrame(item)
        af_df_1 = pd.concat([af_df_1, af_df_temp_1])
    
    
    #global audio_features_tot    
    #for i in range(len(playlist_uris_chunked)):
     #   audio_features_temp = sp.audio_features(tracks=i)
      #  audio_features_tot = audio_features_tot.append(audio_features_temp) #extract audio features 
    
    return (af_df_1) 

song_list = audiofeat_from_playlistLONG ('37i9dQZF1DX55yuR78Invt')

song_list

playlists2 = ['37i9dQZF1DWXRqgorJj26U','3Ho3iO0iJykgEQNbjB2sic','37i9dQZF1DX8ucPPHyzDUE', '37i9dQZF1DX55yuR78Invt', '37i9dQZF1DWWxPM4nWdhyI', '37i9dQZF1DZ06evO161ZV6', '37i9dQZF1DZ06evO2zJOi2', '37i9dQZF1DZ06evO2ZTWSp']

playlists2[0]

audio_tot_df = audiofeat_from_playlistLONG(playlists2[0])

audio_tot_df

for item in playlists2[1:]: #getting the final dataframe
    temp_df = audiofeat_from_playlistLONG(item) 
    #af_df_temp3 = pd.DataFrame(audio_features_playlist)
    audio_tot_df = pd.concat([audio_tot_df, temp_df])

audio_tot_df